In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path

print(Path().cwd())
os.chdir(Path(os.getcwd()).parent)
print(Path().cwd())

/Users/samantha/QuantUS-Plugins-CEUS/TwoD_CEUS_test
/Users/samantha/QuantUS-Plugins-CEUS


## Select Contrast-Enhanced Ultrasound (CEUS) Cine and Parser

In [2]:
from src.image_loading.options import get_scan_loaders

print("Available scan loaders:", list(get_scan_loaders().keys()))

Available scan loaders: ['avi', 'nifti', 'custom_dicom', 'mp4']


In [3]:
scan_type = 'mp4'

scan_path = '/Users/samantha/Desktop/ultrasound lab stuff/mc_ctdna/p14/p14 wk12/p14_wk12.mp4'
scan_loader_kwargs = {
    'transpose': False
}

In [4]:
from src.entrypoints import scan_loading_step

image_data = scan_loading_step(scan_type, scan_path, **scan_loader_kwargs)

## Load Segmentation

Assumes same segmentation for each frame

In [5]:
from src.seg_loading.options import get_seg_loaders

print("Available segmentation loaders:", list(get_seg_loaders().keys()))

Available segmentation loaders: ['nifti', 'load_bolus_mask']


In [6]:
seg_type = 'nifti'

seg_path = '/Users/samantha/Desktop/ultrasound lab stuff/mc_ctdna/p14/p14 wk12/p14_wk12_mc_roi.nii.gz'
seg_loader_kwargs = {}

In [7]:
from src.entrypoints import seg_loading_step

seg_data = seg_loading_step(seg_type, image_data, seg_path, scan_path, **seg_loader_kwargs)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

frames = image_data.pixel_data[:,:,:,0]
seg_mask = seg_data.seg_mask

print(f'Segmentation mask shape: {seg_mask.shape}')
print(f'Image data shape: {image_data.pixel_data.shape}')

frame_slider = widgets.IntSlider(
    value=50,
    min=0,
    max=frames.shape[0] - 1,
    step=1,
    description='Frame',
    continuous_update=False,
    layout=widgets.Layout(width='600px')
)

frame_input = widgets.BoundedIntText(
    value = 50,
    min = 0,
    max = frames.shape[0]-1,
    step = 1,
    description='',
    layout=widgets.Layout(width='80px')
)

overlay_toggle = widgets.ToggleButton(
    value=False,
    description='Show ROI',
    icon='eye',
)

output = widgets.Output()

def _select_seg_frame(frame_idx: int):
    if seg_mask.ndim == 3 and seg_mask.shape[2] == frames.shape[0]:
        return seg_mask[:,:,frame_idx].T
    return seg_mask


def _render_frame(change=None):
    with output:
        output.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(frames[frame_slider.value], cmap='gray')
        if overlay_toggle.value:
            ax.imshow(_select_seg_frame(frame_slider.value), alpha=0.35, cmap='Reds')
        ax.set_title(f"{seg_data.seg_name} - Frame {frame_slider.value}")
        ax.axis('off')
        plt.show()

def _sync_slider_to_input(change):
    frame_input.value = change['new']

def _sync_input_to_slider(change):
    frame_slider.value - change['new']

frame_slider.observe(_render_frame, names='value')
overlay_toggle.observe(_render_frame, names='value')

slider_row = widgets.HBox([frame_slider, overlay_toggle])
# control_row = widgets.HBox([frame_slider, overlay_toggle])
controls = widgets.VBox([slider_row])

display(controls, output)
_render_frame()


Segmentation mask shape: (1962, 1472, 926)
Image data shape: (926, 1472, 1962, 3)


Output()

# Generating T0 map using Segmentation Mask and Pixel data.

In [ ]:
image_data.pixel_data.shape

In [ ]:
from TwoD_CEUS_test.T0_map import generate_t0_map, get_t0_statistics, mask_t0_map

# Generate the T0 map
threshold = 100
start_frame = 80
end_frame = 220

t0_map = generate_t0_map(
    image_data.pixel_data[:,:,:,0],  # Use the first channel if multi-channel
    seg_data.seg_mask,
    threshold=threshold,
    start_frame=start_frame,
    end_frame=end_frame  
)

# Mask the T0 map to show only ROI
masked_t0_map = mask_t0_map(t0_map, seg_data.seg_mask)

# Visualize it
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gamma correction
use_gamma = True      # <--- toggle this to False to disable
gamma = 0.8

if use_gamma:
    t0_map = t0_map ** (1/gamma)
    masked_t0_map = masked_t0_map ** (1/gamma)

# Left: Original with full overlay
single_image = image_data.pixel_data[250,:,:,0]
axes[0].imshow(single_image, cmap='gray')
axes[0].imshow(t0_map, alpha=0.5, cmap='jet')
axes[0].set_title(f'T0 Map - {seg_data.seg_name} | thr {threshold}, frames {start_frame}-{end_frame} (Full Image)')
axes[0].axis('off')

# Right: Masked to show only ROI
axes[1].imshow(single_image, cmap='gray')
axes[1].imshow(masked_t0_map, alpha=0.7, cmap='jet')
axes[1].set_title(f'T0 Map - {seg_data.seg_name} | thr {threshold}, frames {start_frame}-{end_frame} (ROI Only)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# Get statistics
stats = get_t0_statistics(t0_map, seg_data.seg_mask)
print("\nT0 Map Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value:.2f}")